In [34]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier
import tensorflow as tf
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

In [35]:
"""Tworzę nową rankę danych z różnicami między parami (różnica wieku, rasy, jak ważna dla ankietowanej osoby jest wspólna rasa,
różnice w ocenie oraz oczekiwaniach atrakcyjności, szczerości, inteligencji, dowcipu, ambicji, wspólnych zainteresowań,
orelacja wspólnych zainteresowań - dane możliwe do uzyskania podczas ankiety przed randką"""
df = pd.read_csv('speeddating.csv')
new_df = pd.DataFrame()
new_data_list = []
for row in df.itertuples():

    difference_age = row[6]
    difference_race = row[10]
    importance_same_race = row[11]
    difference_attractive_imporatance = abs(row[16]-row[40])
    difference_sincere_imporatance = abs(row[17]-row[41])
    difference_intelligence_imporatance = abs(row[18]-row[42])
    difference_funny_imporatance = abs(row[19]-row[43])
    difference_ambition_imporatance = abs(row[20]-row[44])
    difference_shared_interests_imporatance = abs(row[21]-row[45])
    attractive_difference = abs(row[28] - row[52])
    sinsere_difference = abs(row[29] - row[53])
    intelligence_difference = abs(row[30] - row[54])
    funny_difference = abs(row[31] - row[55])
    ambitous_difference = abs(row[32] - row[56])
    interests_correlation = row[108]
    target = row[123]

    new_data = {
        'difference_age': difference_age,
        'difference_race': difference_race,
        'importance_same_race': importance_same_race,
        'difference_attractive_imporatance': difference_attractive_imporatance,
        'difference_sincere_imporatance': difference_sincere_imporatance,
        'difference_intelligence_imporatance': difference_intelligence_imporatance,
        'difference_funny_imporatance': difference_funny_imporatance,
        'difference_ambition_imporatance': difference_ambition_imporatance,
        'difference_shared_interests_imporatance': difference_shared_interests_imporatance,
        'attractive_difference': attractive_difference,
        'sinsere_difference': sinsere_difference,
        'intelligence_difference': intelligence_difference,
        'funny_difference': funny_difference,
        'ambitous_difference': ambitous_difference,
        'interests_correlation': interests_correlation,
        'target': target
    }
    new_data_list.append(new_data)

new_df = pd.DataFrame(new_data_list)
new_df.to_csv('df_differences.csv', index=False)

In [36]:
"""Sprawdzę jak wyglądają dane"""

'Sprawdzę jak wyglądają dane'

In [37]:
new_df.head()
new_df.describe()
new_df.info()
new_df.isnull().sum()
new_df.dropna(inplace=True)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 16 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   difference_age                           8378 non-null   float64
 1   difference_race                          8378 non-null   int64  
 2   importance_same_race                     8299 non-null   float64
 3   difference_attractive_imporatance        8210 non-null   float64
 4   difference_sincere_imporatance           8210 non-null   float64
 5   difference_intelligence_imporatance      8210 non-null   float64
 6   difference_funny_imporatance             8192 non-null   float64
 7   difference_ambition_imporatance          8174 non-null   float64
 8   difference_shared_interests_imporatance  8130 non-null   float64
 9   attractive_difference                    8069 non-null   float64
 10  sinsere_difference                       7997 no

In [38]:
"""Tworzę pierwszy model - las losowy"""

'Tworzę pierwszy model - las losowy'

In [39]:
X_train, X_test, y_train, y_test = train_test_split(new_df.drop('target', axis=1), new_df['target'], test_size=0.15)
model = RandomForestClassifier(n_estimators=100, max_depth=10)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("accuracy: ", accuracy_score(y_test, y_pred))

accuracy:  0.8497267759562842


In [40]:
"""Dla podanego modelu accuracy wyniosło >0.80, sprawdzę macierz pomyłek"""

'Dla podanego modelu accuracy wyniosło >0.80, sprawdzę macierz pomyłek'

In [41]:
print(confusion_matrix(y_test, y_pred))

[[933   0]
 [165   0]]


In [42]:
"""Jak widać, wynik modelu jest "napompowany" wynikami negatywnymi, nie umie wskazać pozytywnych randek"""
"to pokazuje że accuracy NIE POWINNO być używane do oceny modeli - uczestnik nigdy nie dostanie propozycji randki"
"dla oceny poniższych modeli najważniejsza przyjętą miarą będzie stosunek w ilości sytuacji gdzie model nadał etykietę"
"1 dla danej testowej w której wynikiem faktycznie było 1 do ilości wszystkich danych w których model nadał etykietę 1"
"-chodzi o proponowanie ludziom w pierwszej kolejności takich randek które mają szanse powodzenia i by użytkownik jak najszybciej"
"znalazł odpowiednią osobę"

'znalazł odpowiednią osobę'

In [43]:

for i in range(1,10):
    for j in range(1,10):
        # X_train, X_test, y_train, y_test = train_test_split(new_df.drop('target', axis=1), new_df['target'], test_size=0.15)
        model = RandomForestClassifier(n_estimators=10*i, max_depth=10*j)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print("_____________________")
        print("proba: ", i, " ", j)
        print("confusion matrix: ")
        print(confusion_matrix(y_test, y_pred))
        if confusion_matrix(y_test, y_pred)[1][1]+confusion_matrix(y_test, y_pred)[0][1] == 0:
            print("brak pozytywnych wyników")
        else:
            print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", confusion_matrix(y_test, y_pred)[1][1]/(confusion_matrix(y_test, y_pred)[1][1]+confusion_matrix(y_test, y_pred)[0][1]))
        print("_____________________")
        print("_____________________")

_____________________
proba:  6   4
confusion matrix: 
[[921   3]
 [167   7]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.7
_____________________
_____________________
_____________________
proba:  6   5
confusion matrix: 
[[921   3]
 [167   7]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.7
_____________________
_____________________
_____________________
proba:  6   6
confusion matrix: 
[[918   6]
 [171   3]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.3333333333333333
_____________________
_____________________
_____________________
proba:  6   7
confusion matrix: 
[[923   1]
 [169   5]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.8333333333333334
_____________________
_____________________
_____________________
proba:  6   8
confusion matrix: 
[[919   5]
 [168   6]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.5454545454545454
_____________________
_____________________
__________

In [44]:
"wyniki są dobre dla parametrów n_estimators = 50 i max_depth = 90  , sprawdzę jeszcze kilka razy czy zawsze są one dobre czy tylko raz się udało dla takiego wylosowania danych"
model = RandomForestClassifier(n_estimators=50, max_depth=90)
for i in range(3):
    X_train, X_test, y_train, y_test = train_test_split(new_df.drop('target', axis=1), new_df['target'], test_size=0.15)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("_____________________")
    cm = confusion_matrix(y_test, y_pred)
    print("confusion matrix: ")
    print(cm)
    if cm[1][1]+cm[0][1] == 0:
        print("brak pozytywnych wyników")
    else:
        print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

_____________________
confusion matrix: 
[[919   1]
 [171   7]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.875
_____________________
confusion matrix: 
[[890   2]
 [193  13]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.8666666666666667
_____________________
confusion matrix: 
[[917   1]
 [174   6]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.8571428571428571
_____________________
confusion matrix: 
[[925   4]
 [158  11]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.7333333333333333
_____________________
confusion matrix: 
[[905   2]
 [184   7]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.7777777777777778
_____________________
confusion matrix: 
[[917   0]
 [175   6]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  1.0


In [45]:
"Jako porównanie sprawdzimy jak poradzi sobie nadawanie etykiet losowo, proporcjonalnie do częstości występowania etykiet w danych"
"oraz jakie są szanse gdyby użytkownik miał pójść na randkę z jakąkolwiek osobą"
dummy = DummyClassifier(strategy='stratified')
dummy.fit(X_train, y_train)
y_pred = dummy.predict(X_test)
print("_____________________")
print("losowo ale z proporcjami")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

_____________________
losowo ale z proporcjami
confusion matrix: 
[[749 169]
 [152  28]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.14213197969543148
_____________________
losowo ale z proporcjami
confusion matrix: 
[[758 159]
 [150  31]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.1631578947368421


In [46]:
"Z jednej strony wyniki modelu sąznacznie lepsze, jednak mając zbiór ponad 1000 danych proponuje tylko około" \
"10 randek, co oznacza że aplikacja musiała by mieć bardzo dużą bazę użytkowników, żeby w ogóle działała"
"Teraz dla porównania spróbuję wytrenować sieć neuronową"

'Teraz dla porównania spróbuję wytrenować sieć neuronową'

'Teraz dla porównania spróbuję wytrenować sieć neuronową'

In [47]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["Precision"])

model.fit(X_train, y_train, epochs=5, batch_size=4 , class_weight={0:1, 1:1}, ) #, class_weight=class_weights)


Epoch 1/5
1555/1555 [==============================] - 3s 2ms/step - loss: 0.4388 - precision: 0.3333
Epoch 2/5
1555/1555 [==============================] - 3s 2ms/step - loss: 0.4241 - precision: 0.0000e+00
Epoch 3/5
1555/1555 [==============================] - 3s 2ms/step - loss: 0.4169 - precision: 0.0000e+00
Epoch 4/5
1555/1555 [==============================] - 4s 3ms/step - loss: 0.4122 - precision: 0.6875
Epoch 5/5
1555/1555 [==============================] - 3s 2ms/step - loss: 0.4081 - precision: 0.6000


Epoch 1/5
1555/1555 [==============================] - 3s 1ms/step - loss: 0.4418 - precision: 0.1538
Epoch 2/5
1555/1555 [==============================] - 2s 1ms/step - loss: 0.4231 - precision: 0.1667
Epoch 3/5
1555/1555 [==============================] - 2s 1ms/step - loss: 0.4172 - precision: 0.6667
Epoch 4/5
1555/1555 [==============================] - 2s 1ms/step - loss: 0.4113 - precision: 0.6522
Epoch 5/5
1555/1555 [==============================] - 2s 1ms/step - loss: 0.4071 - precision: 0.6604


In [48]:
y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]
print("_____________________")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

35/35 [==============================] - 0s 2ms/step
_____________________
confusion matrix: 
[[916   2]
 [178   2]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.5
35/35 [==============================] - 0s 948us/step
_____________________
confusion matrix: 
[[914   3]
 [180   1]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.25


In [49]:
"""Sprawdzę jeszcze czy istnieje możliwość stworzenia modelu w oparciu o maszynę wektorów nośnych"""

'Sprawdzę jeszcze czy istnieje możliwość stworzenia modelu w oparciu o maszynę wektorów nośnych'

'Sprawdzę jeszcze czy istnieje możliwość stworzenia modelu w oparciu o maszynę wektorów nośnych'

In [68]:
max = 0
for _ in range(3):
    X_train, X_test, y_train, y_test = train_test_split(new_df.drop('target', axis=1), new_df['target'], test_size=0.15)
    SVM = SVC()
    SVM.fit(X_train, y_train)
    y_pred = SVM.predict(X_test)
    print("_____________________")
    cm = confusion_matrix(y_test, y_pred)
    print("confusion matrix: ")
    print(cm)
    if cm[1][1]+cm[0][1] == 0:
        print("brak pozytywnych wyników")
    else:
        print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))
        if (cm[1][1]/(cm[1][1]+cm[0][1])) > max:
            max = cm[1][1]/(cm[1][1]+cm[0][1])
            best_model = SVM
print("najlepszy model: ", max)

_____________________
confusion matrix: 
[[921   0]
 [177   0]]
brak pozytywnych wyników
_____________________
confusion matrix: 
[[901   0]
 [197   0]]
brak pozytywnych wyników
_____________________
confusion matrix: 
[[916   0]
 [182   0]]
brak pozytywnych wyników
najlepszy model:  0


In [51]:
"""SVM dla przeprowadzonych testów okazałsię najgorszy"""
"""Wyniki dalej nie są zadowalajace, spróbuję teraz jeszcze raz z wcześniejszymi modelami, ale brakujące wartości uzupełnię przez średnią"""

'Wyniki dalej nie są zadowalajace, spróbuję teraz jeszcze raz z wcześniejszymi modelami, ale brakujące wartości uzupełnię przez średnią'

'Wyniki dalej nie są zadowalajace, spróbuję teraz jeszcze raz z wcześniejszymi modelami, ale brakujące wartości uzupełnię przez średnią'

In [52]:
df = pd.read_csv("df_differences.csv")
df = df.fillna(df.mean())
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.15)

In [53]:
"""Random Forest"""

'Random Forest'

'Random Forest'

In [69]:
max = 0
for _ in range(3):
    df = pd.read_csv("df_differences.csv")
    df = df.fillna(df.mean())
    X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.15)

    model = RandomForestClassifier(n_estimators=50, max_depth=90)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("_____________________")
    cm = confusion_matrix(y_test, y_pred)
    print("confusion matrix: ")
    print(cm)
    if cm[1][1]+cm[0][1] == 0:
        print("brak pozytywnych wyników")
    else:
        print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))
        if (cm[1][1]/(cm[1][1]+cm[0][1])) > max:
            max = cm[1][1]/(cm[1][1]+cm[0][1])
print("najlepszy model: ", max)

_____________________
confusion matrix: 
[[1051    3]
 [ 197    6]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.6666666666666666
_____________________
confusion matrix: 
[[1038    6]
 [ 201   12]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.6666666666666666
_____________________
confusion matrix: 
[[1034    3]
 [ 208   12]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.8
najlepszy model:  0.8


In [55]:
"""Po kilku próbach nie udało mi się zaobserwować poprawy wyników"""
"""Nasepnie sprawdzę sieć neuronową"""

'Nasepnie sprawdzę sieć neuronową'

'Nasepnie sprawdzę sieć neuronową'

In [56]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["Precision"])

model.fit(X_train, y_train, epochs=5, batch_size=4 , class_weight={0:1, 1:3}, ) #, class_weight=class_weights)


Epoch 1/5
1781/1781 [==============================] - 3s 1ms/step - loss: 0.8408 - precision: 0.2699
Epoch 2/5
1781/1781 [==============================] - 3s 1ms/step - loss: 0.8106 - precision: 0.2951
Epoch 3/5
1781/1781 [==============================] - 3s 2ms/step - loss: 0.8002 - precision: 0.3135
Epoch 4/5
1781/1781 [==============================] - 3s 1ms/step - loss: 0.7877 - precision: 0.3304
Epoch 5/5
1781/1781 [==============================] - 2s 1ms/step - loss: 0.7805 - precision: 0.3312


Epoch 1/5
1781/1781 [==============================] - 3s 1ms/step - loss: 0.8226 - precision: 0.2757
Epoch 2/5
1781/1781 [==============================] - 2s 1ms/step - loss: 0.7919 - precision: 0.3056
Epoch 3/5
1781/1781 [==============================] - 2s 1ms/step - loss: 0.7786 - precision: 0.3153
Epoch 4/5
1781/1781 [==============================] - 3s 2ms/step - loss: 0.7685 - precision: 0.3331
Epoch 5/5
1781/1781 [==============================] - 2s 1ms/step - loss: 0.7538 - precision: 0.3486


In [57]:
y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]
print("_____________________")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

40/40 [==============================] - 0s 1ms/step
_____________________
confusion matrix: 
[[814 262]
 [ 92  89]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.2535612535612536
40/40 [==============================] - 0s 1ms/step
_____________________
confusion matrix: 
[[895 143]
 [151  68]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.3222748815165877


In [58]:
"""Po kilkukrotnym wyuczeniu modelu z różnymi parametramy również nie zaobserwowałem poprawy, jednak więcej razy model oznacza dane jako pozytywne, więc w przypadku użycia w aplikacji randkowej rzadziej zdarzy się sytuacja, że użytkownikowi nie zostanie zaproponowana randka, która mu odpowiada"""
"""Na koniec sprawdzę jeszcze maszynę wektorów nośnych"""

'Na koniec sprawdzę jeszcze maszynę wektorów nośnych'

'Na koniec sprawdzę jeszcze maszynę wektorów nośnych'

In [70]:
max = 0
for _ in range(3):
    df = pd.read_csv("df_differences.csv")
    df = df.fillna(df.mean())
    X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.15)
    SVM = SVC()
    SVM.fit(X_train, y_train)
    y_pred = SVM.predict(X_test)
    print("_____________________")
    cm = confusion_matrix(y_test, y_pred)
    print("confusion matrix: ")
    print(cm)
    if cm[1][1]+cm[0][1] == 0:
        print("brak pozytywnych wyników")
    else:
        print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))
        if (cm[1][1]/(cm[1][1]+cm[0][1])) > max:
            max = cm[1][1]/(cm[1][1]+cm[0][1])
print("najlepszy model: ", max)

_____________________
confusion matrix: 
[[1038    0]
 [ 219    0]]
brak pozytywnych wyników
_____________________
confusion matrix: 
[[1048    0]
 [ 209    0]]
brak pozytywnych wyników
_____________________
confusion matrix: 
[[1046    0]
 [ 211    0]]
brak pozytywnych wyników
najlepszy model:  0


In [60]:
'''Również nie zaobserwowałem znaczącej poprawy'''
'''Na koniec spróbuję jeszcze poprawić działanie modelu poprzez wyważenie danych metodą oversamplingu'''

'Na koniec spróbuję jeszcze poprawić działanie modelu poprzez wyważenie danych metodą oversamplingu'

'Na koniec spróbuję jeszcze poprawić działanie modelu poprzez wyważenie danych metodą oversamplingu'

In [61]:
df = pd.read_csv("df_differences.csv")
oversampler = SMOTE()
df = df.fillna(df.mean())
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.15)
X_train, y_train = oversampler.fit_resample(X_train, y_train)

In [62]:
max = 0
for _ in range(3):
    df = pd.read_csv("df_differences.csv")
    oversampler = SMOTE()
    df = df.fillna(df.mean())
    X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.15)
    X_train, y_train = oversampler.fit_resample(X_train, y_train)
    model = RandomForestClassifier(n_estimators=50, max_depth=90)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("_____________________")
    cm = confusion_matrix(y_test, y_pred)
    print("confusion matrix: ")
    print(cm)
    if cm[1][1]+cm[0][1] == 0:
        print("brak pozytywnych wyników")
    else:
        print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))
        if (cm[1][1]/(cm[1][1]+cm[0][1])) > max:
            max = cm[1][1]/(cm[1][1]+cm[0][1])
print("najlepszy model: ", max)

_____________________
confusion matrix: 
[[1025   22]
 [ 201    9]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.2903225806451613
_____________________
confusion matrix: 
[[1024   19]
 [ 197   17]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.4722222222222222


In [63]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["Precision"])

model.fit(X_train, y_train, epochs=5, batch_size=4) #, class_weight=class_weights)


Epoch 1/5
2976/2976 [==============================] - 5s 1ms/step - loss: 0.6019 - precision: 0.6498
Epoch 2/5
2976/2976 [==============================] - 4s 1ms/step - loss: 0.5652 - precision: 0.6838
Epoch 3/5
2976/2976 [==============================] - 4s 1ms/step - loss: 0.5426 - precision: 0.6990
Epoch 4/5
2976/2976 [==============================] - 4s 1ms/step - loss: 0.5207 - precision: 0.7212
Epoch 5/5
2976/2976 [==============================] - 4s 1ms/step - loss: 0.5002 - precision: 0.7406


Epoch 1/5
2978/2978 [==============================] - 5s 1ms/step - loss: 0.6030 - precision: 0.6568
Epoch 2/5
2978/2978 [==============================] - 4s 1ms/step - loss: 0.5679 - precision: 0.6887
Epoch 3/5
2978/2978 [==============================] - 4s 1ms/step - loss: 0.5407 - precision: 0.7099
Epoch 4/5
2978/2978 [==============================] - 4s 1ms/step - loss: 0.5191 - precision: 0.7245
Epoch 5/5
2978/2978 [==============================] - 4s 1ms/step - loss: 0.4976 - precision: 0.7393


In [64]:
y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]
print("_____________________")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

40/40 [==============================] - 0s 821us/step
_____________________
confusion matrix: 
[[809 238]
 [126  84]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.2608695652173913
40/40 [==============================] - 0s 1ms/step
_____________________
confusion matrix: 
[[735 308]
 [106 108]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.25961538461538464


In [71]:
for _ in range(3):
    df = pd.read_csv("df_differences.csv")
    oversampler = SMOTE()
    df = df.fillna(df.mean())
    X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.15)
    X_train, y_train = oversampler.fit_resample(X_train, y_train)
    model = SVC()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("_____________________")
    cm = confusion_matrix(y_test, y_pred)
    print("confusion matrix: ")
    print(cm)
    if cm[1][1]+cm[0][1] == 0:
        print("brak pozytywnych wyników")
    else:
        print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))
        if (cm[1][1]/(cm[1][1]+cm[0][1])) > max:
            max = cm[1][1]/(cm[1][1]+cm[0][1])
print("najlepszy model: ", max)

_____________________
confusion matrix: 
[[640 409]
 [ 71 137]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.2509157509157509
_____________________
confusion matrix: 
[[629 410]
 [ 73 145]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.26126126126126126
_____________________
confusion matrix: 
[[665 391]
 [ 74 127]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.24517374517374518
najlepszy model:  0.26126126126126126


In [66]:
"""Ponownie nie zaobserwowałem by któych model mógł stanowić podstawę wyboru partnera"""
"""Do zastosowania w aplikacji najlepiej z przetestowanych modeli nadaje się las losowy, jednak jest on 'oszczędny'"""
"""w ilości przedstawianych rozwiązań, dla 1000 próbek zaledwie przy niektóych losowaniach podziału"""
"""na dane testowe i treningowe czasem mniej niż 10 osób zostaje ze sobą połączonych"""

'na dane testowe i treningowe czasem mniej niż 10 osób zostaje ze sobą połączonych'

'na dane testowe i treningowe czasem mniej niż 10 osób zostaje ze sobą połączonych'